In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime

import pandas as pd
import tentaclio
import sklearn

from phoenix.common import artifacts, run_datetime
from phoenix.common.artifacts.registry_mappers import static_mappers
from phoenix.common import utils
from phoenix.custom_models.tension_classifier import process_annotations
from phoenix.custom_models.tension_classifier.tension_classifier import CountVectorizerTensionClassifier

In [ ]:
# Parameters
# See phoenix/common/run_datetime.py expected format of parameter
RUN_DATETIME = None

# See phoenix/common/artifacts/registry_environment.py expected format of parameter
ARTIFACTS_ENVIRONMENT_KEY = "local"


In [ ]:
if RUN_DATETIME:
    run_dt = run_datetime.from_file_safe_str(RUN_DATETIME)
else:
    run_dt = run_datetime.create_run_datetime_now()
    
url_config = {}
art_url_reg = artifacts.registry.ArtifactURLRegistry(run_dt, ARTIFACTS_ENVIRONMENT_KEY)
STATIC_URL_CUSTOM_MODELS_TENSION_CLASSIFIER_BASE = art_url_reg.get_url("static-custom_models_tension_classifier_base", url_config)
FOLDER_ANNOTATIONS = art_url_reg.get_url("static-custom_models_tension_classifier_data", url_config)

In [ ]:
utils.setup_notebook_output()
utils.setup_notebook_logging()

In [ ]:
# Display params.
print(
STATIC_URL_CUSTOM_MODELS_TENSION_CLASSIFIER_BASE,
FOLDER_ANNOTATIONS,
run_dt.dt,
sep='\n',
)

In [ ]:
# This csv already only has data for the tesnions that we want to classify (and have enough information for.)
df = pd.read_csv(f"{FOLDER_ANNOTATIONS}phoenix_tensions.csv")

In [ ]:
test_df = pd.read_csv(f"{FOLDER_ANNOTATIONS}phoenix_tensions_holdout.csv")

In [ ]:
class_labels = df.filter(like="is_").columns.tolist()

In [ ]:
cv_tension_classifier = CountVectorizerTensionClassifier(class_labels)

In [ ]:
cv_tension_classifier.train(df,test_df, random_state_int=2021)

In [ ]:
cv_tension_classifier

In [ ]:
cv_tension_classifier.analyse()

In [ ]:
cv_tension_classifier.persist_model(STATIC_URL_CUSTOM_MODELS_TENSION_CLASSIFIER_BASE)